<a href="https://colab.research.google.com/github/danirojas-nicole/stonedredliningpubpol2130/blob/main/FINAL_GROUP_Redlining_and_Dispensaries_Version_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%config InlineBackend.figure_formats = ["retina"]
# Makes images appear nice and pretty (this isn't needed but why not)

In [ ]:
!pip install -q geopandas seaborn folium census
# This installs the census package.
!pip install census
# This specifies US from the census package.
!pip install us

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 11.8 MB/s eta 0:00:00


In [ ]:
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from branca.utilities import color_brewer

# Importing census package, import states, make sure pandas library is imported, and import os to make automated functions easier.
from census import Census
from us import states
import os

# Added in for geocoding.
import requests

# Redlining (Izabela, partly Dani)

In [ ]:
census = Census("", year=2020)

In [ ]:
hri_gdf = gpd.read_file("https://raw.githubusercontent.com/PUBPOL-2130/notebooks/refs/heads/main/data/HRI2020.zip")

In [ ]:
hri_gdf

,OBJECTID_1,OBJECTID,GEOID20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,CBSA10,METRO_NAME,Shape_Leng,EQINTER20,Shape_Le_1,Shape_Area,HRI2020,RHRI2020,geometry
0,0,1,01097001502,848502.0,0.0,+30.6568785,-088.0688375,33681.0,"Mobile, AL",0.043989,4,0.043989,0.000080,3.924200,3.9,"POLYGON ((-88.07488 30.64997, -88.07459 30.650..."
1,0,3,01073004502,2953269.0,0.0,+33.5071040,-086.8031176,13820.0,"Birmingham-Hoover, AL",0.077309,4,0.077309,0.000287,3.939468,3.9,"POLYGON ((-86.81747 33.50731, -86.8174 33.5073..."
2,0,4,01073004501,723960.0,0.0,+33.4983380,-086.8072523,13820.0,"Birmingham-Hoover, AL",0.039856,4,0.039856,0.000070,3.551947,3.6,"POLYGON ((-86.81556 33.49672, -86.81517 33.497..."
3,0,6,01073002402,4990907.0,0.0,+33.5299121,-086.7865984,13820.0,"Birmingham-Hoover, AL",0.102104,4,0.102104,0.000484,3.997646,4.0,"POLYGON ((-86.80414 33.52829, -86.80397 33.528..."
4,0,7,01073002401,3488479.0,0.0,+33.5209629,-086.7775618,13820.0,"Birmingham-Hoover, AL",0.101521,4,0.101521,0.000339,3.751425,3.8,"POLYGON ((-86.79666 33.5172, -86.79389 33.5185..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13483,0,16596,55079007900,534569.0,24150.0,+43.0661920,-087.8957527,33340.0,"Milwaukee-Waukesha-West Allis,",0.037109,4,0.037109,0.000062,3.956915,4.0,"POLYGON ((-87.9004 43.06418, -87.90038 43.0652..."
13484,0,16597,55079008000,326024.0,0.0,+43.0674802,-087.9028298,33340.0,"Milwaukee-Waukesha-West Allis,",0.024456,4,0.024456,0.000036,4.000000,4.0,"POLYGON ((-87.90534 43.06421, -87.90533 43.065..."
13485,0,16598,55079007700,679121.0,21023.0,+43.0649318,-087.8886201,33340.0,"Milwaukee-Waukesha-West Allis,",0.038491,3,0.038491,0.000077,3.000000,3.0,"POLYGON ((-87.89505 43.06792, -87.89497 43.067..."
13486,0,16599,55079008100,278881.0,0.0,+43.0674717,-087.9074384,33340.0,"Milwaukee-Waukesha-West Allis,",0.023043,4,0.023043,0.000031,3.650345,3.7,"POLYGON ((-87.90961 43.06425, -87.9096 43.0653..."


In [ ]:
# A list of our areas of interest in NY.
# We'll be working on upstate NY regions within the hri_gdf dataset.
metro_areas = {
    "Syracuse, NY": hri_gdf[hri_gdf["METRO_NAME"] == "Syracuse, NY"],
    "Buffalo-Cheektowaga-Niagara Fa": hri_gdf[hri_gdf["METRO_NAME"] == "Buffalo-Cheektowaga-Niagara Fa"],
    "Rochester, NY": hri_gdf[hri_gdf["METRO_NAME"] == "Rochester, NY"],
    "Poughkeepsie-Newburgh, NY": hri_gdf[hri_gdf["METRO_NAME"] == "Poughkeepsie-Newburgh, NY"],
    "Albany-Schenectady-Troy, NY": hri_gdf[hri_gdf["METRO_NAME"] == "Albany-Schenectady-Troy, NY"],
    "Binghamton, NY": hri_gdf[hri_gdf["METRO_NAME"] == "Binghamton, NY"],
    "Utica-Rose, NY": hri_gdf[hri_gdf["METRO_NAME"] == "Utica-Rose, NY"],
    "Elmira, NY": hri_gdf[hri_gdf["METRO_NAME"] == "Elmira, NY"],
}
# Next, we will filter the redlining data so it only consists of the above metro areas.

In [ ]:
# This makes a list of the metro areas we want.
metro_areas = list(metro_areas.keys())

# This filters the redlining data so it only consists of the above metro areas.
filtered_hri_gdf = hri_gdf[hri_gdf["METRO_NAME"].isin(metro_areas)]

In [ ]:
filtered_hri_gdf

,OBJECTID_1,OBJECTID,GEOID20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,CBSA10,METRO_NAME,Shape_Leng,EQINTER20,Shape_Le_1,Shape_Area,HRI2020,RHRI2020,geometry
7630,0,9416,36067005300,695617.0,0.0,+43.0317144,-076.1476356,45060.0,"Syracuse, NY",0.038292,3,0.038292,0.000077,3.178969,3.2,"POLYGON ((-76.15383 43.03142, -76.15324 43.031..."
7631,0,9417,36067000300,541795.0,0.0,+43.0810832,-076.1483759,45060.0,"Syracuse, NY",0.040518,2,0.040518,0.000060,2.022436,2.0,"POLYGON ((-76.15607 43.08171, -76.15539 43.082..."
7632,0,9418,36067002300,572622.0,0.0,+43.0555187,-076.1483882,45060.0,"Syracuse, NY",0.035841,4,0.035841,0.000063,3.609256,3.6,"POLYGON ((-76.15482 43.05429, -76.15481 43.054..."
7633,0,9419,36067001400,415681.0,0.0,+43.0638377,-076.1481159,45060.0,"Syracuse, NY",0.026849,3,0.026849,0.000046,3.126835,3.1,"POLYGON ((-76.15288 43.06407, -76.15241 43.064..."
7634,0,9422,36067000600,617027.0,0.0,+43.0699860,-076.1526402,45060.0,"Syracuse, NY",0.034675,3,0.034675,0.000068,2.547904,2.5,"POLYGON ((-76.15903 43.06731, -76.15859 43.067..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10027,0,12122,36055003101,647467.0,0.0,+43.1466332,-077.5783349,40380.0,"Rochester, NY",0.040034,3,0.040034,0.000072,2.969962,3.0,"POLYGON ((-77.58721 43.14284, -77.58578 43.143..."
10036,0,12131,36055009605,1202991.0,0.0,+43.1604347,-077.6410866,40380.0,"Rochester, NY",0.070386,4,0.070386,0.000133,3.784735,3.8,"POLYGON ((-77.65414 43.16465, -77.65297 43.164..."
10037,0,12132,36055001600,906982.0,0.0,+43.1728355,-077.5912565,40380.0,"Rochester, NY",0.062798,4,0.062798,0.000100,3.996539,4.0,"POLYGON ((-77.60163 43.16901, -77.60133 43.169..."
10094,0,12197,36093020102,854905.0,0.0,+42.8295375,-073.9161408,10580.0,"Albany-Schenectady-Troy, NY",0.057838,3,0.057838,0.000094,3.008394,3.0,"POLYGON ((-73.92513 42.82029, -73.9244 42.8214..."


# Dispensaries (Izabela)

In [ ]:
# To load in the dispensary data, drag the OCM2.csv from your downloads into the Files tab.
disp_df = pd.read_csv("OCM.csv")

In [ ]:
disp_df

,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,Application Number,SEE Category,...,Processing Activities Blending and Infusing,Processing Activities Packaging and Labeling,Processing Activities Branding,Retail Activities Sales with Delivery,Retail Activities Sales No Delivery,Retail Activities Non-Cannabis Products,Retail Activities Drive Thru,Retail Date Opened to Public,Hours of Operation,Primary Contact Name
0,OCM-RETL-25-000306,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,03/24/2025 12:00:00 AM,03/24/2025 12:00:00 AM,03/24/2027 12:00:00 AM,OCMRETL-2023-000090,"Women-Owned Business, Minority-Owned Business,...",...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,Jennifer Babaian
1,OCM-PROC-24-000096,Adult-Use Processor License,OCMPROC,Active,LICACT,09/12/2024 12:00:00 AM,09/12/2024 12:00:00 AM,09/12/2026 12:00:00 AM,OCMPROC-2023-000306,Women-Owned Business,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,Lori Gronman
2,OCM-RETL-24-000123,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/10/2024 12:00:00 AM,07/10/2024 12:00:00 AM,07/10/2026 12:00:00 AM,OCMRETL-2023-001433,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,Jon-Eric Aben
3,OCM-RETL-25-000303,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,02/25/2025 12:00:00 AM,02/25/2025 12:00:00 AM,02/25/2027 12:00:00 AM,OCMRETL-2023-001318,"Women-Owned Business, Minority-Owned Business",...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,Avie Li
4,OCM-CAURD-24-000177,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,08/02/2024 12:00:00 AM,08/02/2024 12:00:00 AM,08/02/2026 12:00:00 AM,OCMCAURDP-2023-000228,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11/19/2024 12:00:00 AM,Sun: 10:00 AM - 10:00 PM; Mon: 10:00 AM - 10:0...,JOHN GUERCIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261,OCM-DIST-24-000053,Adult-Use Distributor License,OCMDIST,Active,LICACT,07/24/2024 12:00:00 AM,07/24/2024 12:00:00 AM,07/24/2026 12:00:00 AM,OCMDIST-2023-000219,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,Zion Foss
2262,OCM-PROC-24-000075,Adult-Use Processor License,OCMPROC,Active,LICACT,07/23/2024 12:00:00 AM,07/23/2024 12:00:00 AM,07/23/2026 12:00:00 AM,OCMPROC-2023-000301,NaN,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,Zion Foss
2263,OCM-PROC-24-000178,Adult-Use Processor License,OCMPROC,Active,LICACT,10/30/2024 12:00:00 AM,10/30/2024 12:00:00 AM,10/30/2026 12:00:00 AM,OCMPROC-2023-000170,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,Salvatore Pettinato
2264,OCM-RETL-24-000143,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/17/2024 12:00:00 AM,07/17/2024 12:00:00 AM,07/17/2026 12:00:00 AM,OCMRETL-2023-000298,Minority-Owned Business,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,Raman Singh


In [ ]:
# Here we're dropping any columns from the dataset that we won't use.
# We're keeping the License Number to do geocoding.
filtered_disp_df = disp_df.drop(["Application Number", "Location ID", "Business Website",
                               "Tier Type", "Processor Type", "Cultivation Indoor", "Cultivation Outdoor",
                               "Cultivation Mixed Light", "Cultivation Combination", "Cultivation Activities Drying Curing",
                               "Cultivation Activities Storage", "Cultivation Activities Packaging",
                               "Cultivation Activities Waste Rendering", "Processing Activities Extraction",
                               "Processing Activities Blending and Infusing", "Processing Activities Packaging and Labeling",
                               "Processing Activities Branding"], axis=1)

In [ ]:
# Filtering through the data to only look at "Active" licenses.
filtered_disp_df = filtered_disp_df[(filtered_disp_df["License Status"] == "Active")]
#filtered_disp_df = filtered_disp_df[(filtered_disp_df["Operational Status"] == "Active")]
filtered_disp_df

,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,SEE Category,Entity Name,...,Longitude,Operational Status,Business Purpose,Retail Activities Sales with Delivery,Retail Activities Sales No Delivery,Retail Activities Non-Cannabis Products,Retail Activities Drive Thru,Retail Date Opened to Public,Hours of Operation,Primary Contact Name
0,OCM-RETL-25-000306,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,03/24/2025 12:00:00 AM,03/24/2025 12:00:00 AM,03/24/2027 12:00:00 AM,"Women-Owned Business, Minority-Owned Business,...",100 North 3rd Ltd,...,-73.962769,Non-Operational,Adult-Use Retail Sales,1.0,0.0,1.0,0.0,NaN,NaN,Jennifer Babaian
1,OCM-PROC-24-000096,Adult-Use Processor License,OCMPROC,Active,LICACT,09/12/2024 12:00:00 AM,09/12/2024 12:00:00 AM,09/12/2026 12:00:00 AM,Women-Owned Business,10395727 NY LLC,...,NaN,Active,Adult-Use Processing,0.0,0.0,0.0,0.0,NaN,NaN,Lori Gronman
2,OCM-RETL-24-000123,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/10/2024 12:00:00 AM,07/10/2024 12:00:00 AM,07/10/2026 12:00:00 AM,NaN,10415 Jamaica LLC,...,-74.386720,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Jon-Eric Aben
3,OCM-RETL-25-000303,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,02/25/2025 12:00:00 AM,02/25/2025 12:00:00 AM,02/25/2027 12:00:00 AM,"Women-Owned Business, Minority-Owned Business",10613 Northern LLC,...,-73.862295,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Avie Li
4,OCM-CAURD-24-000177,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,08/02/2024 12:00:00 AM,08/02/2024 12:00:00 AM,08/02/2026 12:00:00 AM,NaN,1106 REBECCA ST LIMITED LIABILITY COMPANY,...,NaN,Active,Adult-Use Retail Dispensary,0.0,1.0,0.0,0.0,11/19/2024 12:00:00 AM,Sun: 10:00 AM - 10:00 PM; Mon: 10:00 AM - 10:0...,JOHN GUERCIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,OCM-CULT-24-000079,Adult-Use Cultivator License,OCMCULT,Active,LICACT,04/12/2024 12:00:00 AM,04/12/2024 12:00:00 AM,04/12/2026 12:00:00 AM,NaN,ZIZ NY GRW LLC,...,NaN,Active,Adult-Use Cultivation,0.0,0.0,0.0,0.0,NaN,NaN,Zion Foss
2261,OCM-DIST-24-000053,Adult-Use Distributor License,OCMDIST,Active,LICACT,07/24/2024 12:00:00 AM,07/24/2024 12:00:00 AM,07/24/2026 12:00:00 AM,NaN,ZIZ NY GRW LLC,...,NaN,Active,Adult-Use Distribution,0.0,0.0,0.0,0.0,NaN,NaN,Zion Foss
2262,OCM-PROC-24-000075,Adult-Use Processor License,OCMPROC,Active,LICACT,07/23/2024 12:00:00 AM,07/23/2024 12:00:00 AM,07/23/2026 12:00:00 AM,NaN,ZIZ NY GRW LLC,...,NaN,Active,Adult-Use Processing,0.0,0.0,0.0,0.0,NaN,NaN,Zion Foss
2263,OCM-PROC-24-000178,Adult-Use Processor License,OCMPROC,Active,LICACT,10/30/2024 12:00:00 AM,10/30/2024 12:00:00 AM,10/30/2026 12:00:00 AM,NaN,Zooted SI LLC,...,NaN,Active,Adult-Use Processing,0.0,0.0,0.0,0.0,NaN,NaN,Salvatore Pettinato


In [ ]:
# Since this data is all from New York anyway, we'll set the state to New York to account for any unfilled values.
filtered_disp_df['State'] = 'New York'

In [ ]:
# Looks at all of the Address Line 1 values and Address Line 2 values.
#filtered_disp_df[filtered_disp_df['Address Line 2'].notna()][['Address Line 1', 'Address Line 2']]

In [ ]:
# The amount of Address Line 2 values that aren't na.
#filtered_disp_df['Address Line 2'].notna().sum()

In [ ]:
# Rename and subset columns.
clean_disp_df = filtered_disp_df.rename(
    columns={"License Number" : "Unique ID", "Address Line 1" : "Street Address",
             "city": "City",
             "Zip Code": "ZIP"}
)[["Unique ID", "Street Address", "City", "State", "ZIP"]]

clean_disp_df["Street Address"] = clean_disp_df["Street Address"].str.replace(
    ",", "-"
)

# View our data.
clean_disp_df = clean_disp_df[clean_disp_df['Street Address'].notna()]
clean_disp_df.head()

,Unique ID,Street Address,City,State,ZIP
0,OCM-RETL-25-000306,100 N 3rd St,Brooklyn,New York,11249.0
2,OCM-RETL-24-000123,172 State Route 94 South,Warwick,New York,10990.0
3,OCM-RETL-25-000303,10613 Northern Blvd,Corona,New York,11368.0
4,OCM-CAURD-24-000177,304 Ellery St,Brooklyn,New York,11206.0
5,OCM-RETL-24-000109,345 E 115th St,New York,New York,10029.0


In [ ]:
# Save to local CSV for geocoding.
clean_disp_df.to_csv("addresses.csv", index=False)

In [ ]:
# The geocoding step.
url = "https://geocoding.geo.census.gov/geocoder/geographies/addressbatch"
parameters = {
    "benchmark": "Public_AR_Current",
    "vintage": "Current_Current",
    "format": "json",
}
files = {"addressFile": ("addresses.csv", open(r"./addresses.csv", "rb"))}
r = requests.post(url, files=files, data=parameters)

In [ ]:
# Need to clean the geocoded result, which is just a text string.
# Note: Results do not have headers, so we will have to name the columns ourselves via the documentation.
geocoded_lines_list = [
    x.replace('"', "").strip().split(",") for x in r.text.splitlines()
]
geocoded_disp_df = pd.DataFrame(
    geocoded_lines_list,
    columns=[
        "Record ID Number",
        "Input Address",
        "Input City",
        "Input State",
        "Input Zip",
        "Match Indicator",
        "TIGER Match Type",
        "TIGER Output Street Address",
        "TIGER Output City",
        "TIGER Output State",
        "TIGER Output Zip",
        "Interpolated Longitude",
        "Interpolated Latitude",
        "Tigerline ID",
        "Tigerline ID Side",
        "State",
        "County",
        "Tract",
        "Block",
    ],
)
geocoded_disp_df

,Record ID Number,Input Address,Input City,Input State,Input Zip,Match Indicator,TIGER Match Type,TIGER Output Street Address,TIGER Output City,TIGER Output State,TIGER Output Zip,Interpolated Longitude,Interpolated Latitude,Tigerline ID,Tigerline ID Side,State,County,Tract,Block
0,MM0103D,642 Old Liverpool Rd.,Liverpool,New York,13088.0,Match,Non_Exact,642 OLD LIVERPOOL RD,LIVERPOOL,NY,13088,-76.193267424081,43.092433933422,46564601,R,36,067,013701,3009
1,OCM-CAURD-24-000090,650 Orchard Park Rd,West Seneca,New York,14224.0,Match,Non_Exact,650 ORCHARD PARK RD,WEST SENECA,NY,14224,-78.776700043979,42.830671809037,48344977,L,36,029,011700,3023
2,OCM-RETL-24-000220,24502 Horace Harding Expy,Little Neck,New York,11362.0,Match,Non_Exact,245-02 HORACE HARDING EXPY,LITTLE NECK,NY,11362,-73.737156877395,40.7578928918,59726872,R,36,081,152901,3001
3,OCM-CAURD-24-000091,118 Flatbush Ave,Brooklyn,New York,11217.0,Match,Non_Exact,118 FLATBUSH AVE,BROOKLYN,NY,11217,-73.978152146497,40.684911676885,59077595,R,36,047,003900,1003
4,OCM-RETL-24-000222,1031 Southern Blvd,Bronx,New York,10459.0,Match,Non_Exact,1031 SOUTHERN BLVD,BRONX,NY,10459,-73.891934682536,40.823870754894,80302390,L,36,005,015900,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,OCM-RETL-24-000169,8137 Lefferts Blvd,Kew Gardens,New York,11415.0,Match,Non_Exact,81-37 LEFFERTS BLVD,KEW GARDENS,NY,11415,-73.830628853497,40.708465358412,59733949,L,36,081,013600,1001
725,OCM-RETL-24-000048,1536 Amsterdam Ave,New York,New York,10031.0,Match,Non_Exact,1536 AMSTERDAM AVE,NEW YORK,NY,10031,-73.952062216134,40.819261050766,59655897,L,36,061,022301,2002
726,OCM-RETL-24-000168,37 N Main St,Ellenville,New York,12428.0,Match,Non_Exact,37 N MAIN ST,ELLENVILLE,NY,12428,-74.392120090878,41.721418855444,41872083,L,36,111,954800,3007
727,OCM-RETL-24-000047,217-12 Hempstead Ave,Queens Village,New York,11429.0,Match,Non_Exact,217-12 HEMPSTEAD AVE,QUEENS VILLAGE,NY,11429,-73.740898244036,40.714736301338,59717990,R,36,081,054000,3004


In [ ]:
# Merges the filtered and geocoded data together.
geocoded_mgd_disp_df = filtered_disp_df[filtered_disp_df['Latitude'].isna()&filtered_disp_df['Address Line 1'].notna()].merge(geocoded_disp_df.drop(columns=['State', 'County']), left_on='License Number',
                                right_on='Record ID Number', how='inner')

In [ ]:
#filtered_disp_df[filtered_disp_df['Latitude'].isna()&filtered_disp_df['Address Line 1'].notna()].shape, geocoded_mgd_disp_df.shape

In [ ]:
# The amount of merged geocoded data that didn't have a matching location for its address.
(geocoded_mgd_disp_df['Match Indicator'] == 'No_Match').sum()

np.int64(8)

In [ ]:
# The geocoded data that didn't have a matching location for its address.
geocoded_mgd_disp_df[geocoded_mgd_disp_df['Match Indicator']=='No_Match'][['Address Line 1', 'Input Address', 'Input City', 'Zip Code']]

,Address Line 1,Input Address,Input City,Zip Code
48,475 Central Ave,475 Central Ave,White Plains,10606.0
67,2075 Central Ave,2075 Central Ave,Schenectady,12304.0
72,2988 US Route 20,2988 US Route 20,Seneca Falls,13148.0
95,2532 Route 9N,2532 Route 9N,Greenfield Center,12833.0
102,8047-8053 Pittsford-Victor Road,8047-8053 Pittsford-Victor Road,Victor,14564.0
172,308 Plaza Rd,308 Plaza Rd,Kingston,12401.0
186,228 Route 109,228 Route 109,Farmingdale,11735.0
250,9200 Niagara Falls Blvd,9200 Niagara Falls Blvd,Niagara Falls,14304.0


In [ ]:
# Manually geocoding the above rows.
# 475 Central Ave [48]
geocoded_mgd_disp_df.loc[48, "Interpolated Latitude"] = 41.029936
geocoded_mgd_disp_df.loc[48, "Interpolated Longitude"] = -73.788703
# 2075 Central Ave [67]
geocoded_mgd_disp_df.loc[67, "Interpolated Latitude"] = 42.751428
geocoded_mgd_disp_df.loc[67, "Interpolated Longitude"] = -73.870721
# 2988 US Route 20 [72]
geocoded_mgd_disp_df.loc[72, "Interpolated Latitude"] = 42.94971425
geocoded_mgd_disp_df.loc[72, "Interpolated Longitude"] = -76.76649432
# 2532 Route 9N [95]
geocoded_mgd_disp_df.loc[95, "Interpolated Latitude"] = 43.123522
geocoded_mgd_disp_df.loc[95, "Interpolated Longitude"] = -73.846921
# Pittsford-Victor Road [102]
geocoded_mgd_disp_df.loc[102, "Interpolated Latitude"] = 43.039521
geocoded_mgd_disp_df.loc[102, "Interpolated Longitude"] = -77.458279
# 308 Plaza Rd [172]
geocoded_mgd_disp_df.loc[172, "Interpolated Latitude"] = 41.93798411
geocoded_mgd_disp_df.loc[172, "Interpolated Longitude"] = -74.01949019

# 228 Route 109 [186]: WARNING
# This piece of data looks suspicious. We looked it up on streetview and there didn't seem
# to be a cannabis store there. Hence, we're not going to add latitude and longitude coordinates to it.
# geocoded_mgd_disp_df.loc[186, "Interpolated Latitude"] = 40.72281501
# geocoded_mgd_disp_df.loc[186, "Interpolated Longitude"] = -73.42396373

# 9200 Niagara Falls Blvd [250]
geocoded_mgd_disp_df.loc[250, "Interpolated Latitude"] = 43.09446896
geocoded_mgd_disp_df.loc[250, "Interpolated Longitude"] = -78.95678494

In [ ]:
# In addition to looking suspicious, this dispo has an active license but has a "non operational" operative status
geocoded_mgd_disp_df.loc[186]

,186
License Number,OCM-CAURD-24-000168
License Type,Adult-Use Conditional Retail Dispensary License
License Type Code,OCMCAURD22
License Status,Active
License Status Code,LICACT
Issued Date,07/12/2024 12:00:00 AM
Effective Date,07/12/2024 12:00:00 AM
Expiration Date,07/12/2026 12:00:00 AM
SEE Category,NaN
Entity Name,NYCC 1 LLC


In [ ]:
# Renames some columns and gets rid of missing values for the addresses.
geo_add_disp_df = geocoded_mgd_disp_df[geocoded_mgd_disp_df['Longitude'].isna()&geocoded_mgd_disp_df['Latitude'].isna()\
                     &geocoded_mgd_disp_df['Address Line 1'].notna()]

geo_add_disp_df = geo_add_disp_df.drop(columns=['Latitude', 'Longitude']).rename(columns={'Interpolated Longitude':'Longitude',
                                     'Interpolated Latitude':'Latitude'})[filtered_disp_df.columns]

In [ ]:
# Gets rid of missing values for the latitude and longitude for the filtered_disp_df.
nonmissing_filtered_disp_df = filtered_disp_df[filtered_disp_df['Longitude'].notna()&filtered_disp_df['Latitude'].notna()]

In [ ]:
complete_filtered_disp_df = pd.concat([nonmissing_filtered_disp_df, geo_add_disp_df], axis=0)
complete_filtered_disp_df

,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,SEE Category,Entity Name,...,Longitude,Operational Status,Business Purpose,Retail Activities Sales with Delivery,Retail Activities Sales No Delivery,Retail Activities Non-Cannabis Products,Retail Activities Drive Thru,Retail Date Opened to Public,Hours of Operation,Primary Contact Name
0,OCM-RETL-25-000306,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,03/24/2025 12:00:00 AM,03/24/2025 12:00:00 AM,03/24/2027 12:00:00 AM,"Women-Owned Business, Minority-Owned Business,...",100 North 3rd Ltd,...,-73.962769,Non-Operational,Adult-Use Retail Sales,1.0,0.0,1.0,0.0,NaN,NaN,Jennifer Babaian
2,OCM-RETL-24-000123,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/10/2024 12:00:00 AM,07/10/2024 12:00:00 AM,07/10/2026 12:00:00 AM,NaN,10415 Jamaica LLC,...,-74.38672,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Jon-Eric Aben
3,OCM-RETL-25-000303,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,02/25/2025 12:00:00 AM,02/25/2025 12:00:00 AM,02/25/2027 12:00:00 AM,"Women-Owned Business, Minority-Owned Business",10613 Northern LLC,...,-73.862295,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Avie Li
5,OCM-RETL-24-000109,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,06/21/2024 12:00:00 AM,06/21/2024 12:00:00 AM,06/21/2026 12:00:00 AM,Minority-Owned Business,115 Corner LLC,...,-73.93629,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Isa Brnja
8,OCM-RETL-25-000304,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,02/25/2025 12:00:00 AM,02/25/2025 12:00:00 AM,02/25/2027 12:00:00 AM,Minority-Owned Business,14707 45th LLC,...,-73.817641,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Wei Li
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,OCM-CAURD-23-000038,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,12/21/2023 12:00:00 AM,12/21/2023 12:00:00 AM,12/21/2025 12:00:00 AM,NaN,"WhiteboxTHC, LLC",...,-73.957066079612,Active,Adult-Use Retail Dispensary,0.0,1.0,0.0,0.0,01/02/2024 12:00:00 AM,Sun: 10:00 AM - 08:00 PM; Mon: 10:00 AM - 10:0...,Wei Hu
261,OCM-CAURD-23-000006,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,03/10/2023 12:00:00 AM,03/10/2023 12:00:00 AM,03/10/2025 12:00:00 AM,NaN,WilliamJane Corporation,...,-76.498561995988,Active,Adult-Use Retail Dispensary,0.0,0.0,0.0,0.0,05/16/2024 12:00:00 AM,Sun: 11:00 AM - 07:00 PM; Mon: 10:00 AM - 10:0...,william durham
262,OCM-CAURD-24-000085,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,02/20/2024 12:00:00 AM,02/20/2024 12:00:00 AM,02/20/2026 12:00:00 AM,NaN,"WNYC II, LLC",...,-78.800326867786,Active,Adult-Use Retail Dispensary,0.0,1.0,0.0,1.0,02/26/2024 12:00:00 AM,Sun: 11:00 AM - 05:00 PM; Mon: 09:00 AM - 09:0...,Johnny Chebat
263,OCM-CAURD-24-000145,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,05/17/2024 12:00:00 AM,05/17/2024 12:00:00 AM,05/17/2026 12:00:00 AM,NaN,"XANDER LEGACY, LLC",...,-73.963122652414,Active,Adult-Use Retail Dispensary,0.0,1.0,0.0,0.0,11/15/2024 12:00:00 AM,Sun: 10:00 AM - 10:00 PM; Mon: 10:00 AM - 10:0...,THEODORE CRAWFORD


In [ ]:
# Using the Longitude and Latitude coordinates to create points for the spatial join.
# Note: This is a gdf, not a df, since this is working with geospatial data.
final_disp_gdf = gpd.GeoDataFrame(complete_filtered_disp_df, geometry=gpd.points_from_xy(complete_filtered_disp_df["Longitude"], complete_filtered_disp_df["Latitude"]))

In [ ]:
final_disp_gdf

,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,SEE Category,Entity Name,...,Operational Status,Business Purpose,Retail Activities Sales with Delivery,Retail Activities Sales No Delivery,Retail Activities Non-Cannabis Products,Retail Activities Drive Thru,Retail Date Opened to Public,Hours of Operation,Primary Contact Name,geometry
0,OCM-RETL-25-000306,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,03/24/2025 12:00:00 AM,03/24/2025 12:00:00 AM,03/24/2027 12:00:00 AM,"Women-Owned Business, Minority-Owned Business,...",100 North 3rd Ltd,...,Non-Operational,Adult-Use Retail Sales,1.0,0.0,1.0,0.0,NaN,NaN,Jennifer Babaian,POINT (-73.96277 40.7167)
2,OCM-RETL-24-000123,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/10/2024 12:00:00 AM,07/10/2024 12:00:00 AM,07/10/2026 12:00:00 AM,NaN,10415 Jamaica LLC,...,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Jon-Eric Aben,POINT (-74.38672 41.23645)
3,OCM-RETL-25-000303,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,02/25/2025 12:00:00 AM,02/25/2025 12:00:00 AM,02/25/2027 12:00:00 AM,"Women-Owned Business, Minority-Owned Business",10613 Northern LLC,...,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Avie Li,POINT (-73.8623 40.75804)
5,OCM-RETL-24-000109,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,06/21/2024 12:00:00 AM,06/21/2024 12:00:00 AM,06/21/2026 12:00:00 AM,Minority-Owned Business,115 Corner LLC,...,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Isa Brnja,POINT (-73.93629 40.7957)
8,OCM-RETL-25-000304,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,02/25/2025 12:00:00 AM,02/25/2025 12:00:00 AM,02/25/2027 12:00:00 AM,Minority-Owned Business,14707 45th LLC,...,Non-Operational,Adult-Use Retail Sales,1.0,0.0,0.0,0.0,NaN,NaN,Wei Li,POINT (-73.81764 40.75565)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,OCM-CAURD-23-000038,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,12/21/2023 12:00:00 AM,12/21/2023 12:00:00 AM,12/21/2025 12:00:00 AM,NaN,"WhiteboxTHC, LLC",...,Active,Adult-Use Retail Dispensary,0.0,1.0,0.0,0.0,01/02/2024 12:00:00 AM,Sun: 10:00 AM - 08:00 PM; Mon: 10:00 AM - 10:0...,Wei Hu,POINT (-73.95707 40.76908)
261,OCM-CAURD-23-000006,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,03/10/2023 12:00:00 AM,03/10/2023 12:00:00 AM,03/10/2025 12:00:00 AM,NaN,WilliamJane Corporation,...,Active,Adult-Use Retail Dispensary,0.0,0.0,0.0,0.0,05/16/2024 12:00:00 AM,Sun: 11:00 AM - 07:00 PM; Mon: 10:00 AM - 10:0...,william durham,POINT (-76.49856 42.43949)
262,OCM-CAURD-24-000085,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,02/20/2024 12:00:00 AM,02/20/2024 12:00:00 AM,02/20/2026 12:00:00 AM,NaN,"WNYC II, LLC",...,Active,Adult-Use Retail Dispensary,0.0,1.0,0.0,1.0,02/26/2024 12:00:00 AM,Sun: 11:00 AM - 05:00 PM; Mon: 09:00 AM - 09:0...,Johnny Chebat,POINT (-78.80033 43.03464)
263,OCM-CAURD-24-000145,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,05/17/2024 12:00:00 AM,05/17/2024 12:00:00 AM,05/17/2026 12:00:00 AM,NaN,"XANDER LEGACY, LLC",...,Active,Adult-Use Retail Dispensary,0.0,1.0,0.0,0.0,11/15/2024 12:00:00 AM,Sun: 10:00 AM - 10:00 PM; Mon: 10:00 AM - 10:0...,THEODORE CRAWFORD,POINT (-73.96312 40.68263)


# Joined Data (Izabela, partly Dani)

In [ ]:
# Joins together the redlining dataframe and the dispensary dataframe.
joined_gdfs = final_disp_gdf.sjoin(filtered_hri_gdf, how='left')
# Drops the missing data in the HRI2020 subset.
joined_gdf_dropped = joined_gdfs.dropna(subset="HRI2020")
joined_gdf_dropped

/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:2391: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4269

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,SEE Category,Entity Name,...,INTPTLAT20,INTPTLON20,CBSA10,METRO_NAME,Shape_Leng,EQINTER20,Shape_Le_1,Shape_Area,HRI2020,RHRI2020
9,OCM-RETL-24-000138,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/16/2024 12:00:00 AM,07/16/2024 12:00:00 AM,07/16/2026 12:00:00 AM,NaN,1650 CRANE STREET DISPENSARY LLC,...,+42.7935751,-073.9599003,10580.0,"Albany-Schenectady-Troy, NY",0.084889,4.0,0.084889,0.000168,3.276777,3.3
27,OCM-RETL-24-000082,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,"Minority-Owned Business, Women-Owned Business",3807 harlem Cannabis LLC,...,+42.9463750,-078.7871019,15380.0,Buffalo-Cheektowaga-Niagara Fa,0.067753,2.0,0.067753,0.000124,1.976500,2.0
102,OCM-MICR-24-000187,Adult-Use Microbusiness License,OCMMICR,Active,LICACT,11/15/2024 12:00:00 AM,11/15/2024 12:00:00 AM,11/15/2026 12:00:00 AM,Service-Disabled Veteran-Owned Business,AJ'S NATIVE WHOLESALE INC,...,+43.0730009,-078.9694897,15380.0,Buffalo-Cheektowaga-Niagara Fa,0.102452,2.0,0.102452,0.000402,2.108906,2.1
163,OCM-RETL-25-000328,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,04/09/2025 12:00:00 AM,04/09/2025 12:00:00 AM,04/09/2027 12:00:00 AM,NaN,Astrid Holdings LLC,...,+42.9178373,-078.8825681,15380.0,Buffalo-Cheektowaga-Niagara Fa,0.032281,3.0,0.032281,0.000053,2.558066,2.6
172,OCM-RETL-25-000332,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,04/23/2025 12:00:00 AM,04/23/2025 12:00:00 AM,04/23/2027 12:00:00 AM,Minority-Owned Business,Azzam Properties LLC,...,+43.1445440,-077.5945403,40380.0,"Rochester, NY",0.033792,4.0,0.033792,0.000049,3.949353,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,OCM-CAURD-23-000011,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,06/02/2023 12:00:00 AM,06/02/2023 12:00:00 AM,06/02/2025 12:00:00 AM,NaN,Stage One Cannabis LLC,...,+42.6485604,-073.7369113,10580.0,"Albany-Schenectady-Troy, NY",0.092971,4.0,0.092971,0.000196,3.895442,3.9
234,OCM-CAURD-24-000068,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,01/26/2024 12:00:00 AM,01/26/2024 12:00:00 AM,01/26/2026 12:00:00 AM,NaN,"The BonTemps Firm Retail, LLC",...,+42.8991992,-078.8784310,15380.0,Buffalo-Cheektowaga-Niagara Fa,0.034698,3.0,0.034698,0.000056,3.000000,3.0
242,OCM-CAURD-24-000181,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,08/02/2024 12:00:00 AM,08/02/2024 12:00:00 AM,08/02/2026 12:00:00 AM,NaN,The Weed Spot LLC,...,+42.8128925,-073.9389461,10580.0,"Albany-Schenectady-Troy, NY",0.028456,4.0,0.028456,0.000048,3.995970,4.0
250,OCM-CAURD-24-000147,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,05/17/2024 12:00:00 AM,05/17/2024 12:00:00 AM,05/17/2026 12:00:00 AM,NaN,Upstate Exotics LLC,...,+43.0929816,-078.9512927,15380.0,Buffalo-Cheektowaga-Niagara Fa,0.056720,2.0,0.056720,0.000152,2.000000,2.0


In [ ]:
joined_gdfs["County"].unique()
joined_gdfs[joined_gdfs["HRI2020"].isna()]["County"].unique()

array(['Kings', 'Orange', 'Queens', 'New York', 'Rensselaer', 'Rockland',
       'Erie', 'Dutchess', 'Sullivan', 'Richmond', 'Ulster', 'Suffolk',
       'Jefferson', 'Niagara', 'Westchester', 'Cayuga', 'Tompkins',
       'St Lawrence', 'Bronx', 'Schuyler', 'Wayne', 'Schoharie', 'Oswego',
       'Chautauqua', 'Genesee', 'Oneida', 'Saratoga', 'Delaware',
       'Greene', 'Albany', 'Cattaraugus', 'Cortland', 'Steuben',
       'Clinton', 'Nassau', 'Livingston', 'Broome', 'Essex', 'Warren',
       'Monroe', 'Fulton', 'Columbia', 'Chenango', 'Franklin', 'Seneca',
       'Montgomery', 'Tioga', 'Otsego', 'Schenectady', 'Washington',
       'Onondaga', 'Lewis', 'Herkimer', 'Ontario', 'Chemung', 'Yates'],
      dtype=object)

In [ ]:
# The HRI2020 data that was dropped above.
# It SEEMS like many of these counties are in the city
joined_gdfs[joined_gdfs["HRI2020"].isna()]["County"]

,County
0,Kings
2,Orange
3,Queens
5,New York
8,Queens
...,...
260,New York
261,Tompkins
262,Erie
263,Kings


In [ ]:
# We're not looking at NYC counties, so we filter them out here.
# nyc_counties = ["New York", "Kings", "Bronx", "Richmond", "Queens"]
# filtered_counties = joined_gdfs[joined_gdfs["HRI2020"].isna()]["County"][
#     ~joined_gdfs[joined_gdfs["HRI2020"].isna()]["County"].isin(nyc_counties)
# ]

In [ ]:
# Non-NYC counties that we need the FIPS code for the ACS data.
# filtered_counties

Note to Izabela: Do a map of the number of dispensaries by counties in NY state (make a chloropleth)

# Prepping for ACS and other comparisons, getting county FIPS codes (Dani)

In [ ]:
# We're going to pull from ACS 5 year estimates, 2019 - 2023. Since NY state OCM was founded in 2021,
# We're going to compare ACS info from 2019 and 2023 (before vs after).

[Scraped FIPS codes](http://unicede.air-worldwide.com/unicede/unicede_new-york_fips_3.html) <-- Generate a dataframe of all NY state county fips code to filter by the applicable counties

In [ ]:
# Need FIPS of NY counties, import that df, then filter, refer to w3 notebook for assistance.

In [ ]:
ny_county_fips_df = pd.read_csv("/content/ny county fips codes (scraped from unicde website with data scraper extension).csv")

In [ ]:
# P-category is fips code, three digits per entry, add zeros in front if need be.
ny_county_fips_df

,p,p 2
0,1,Albany
1,3,Allegany
2,5,Bronx
3,7,Broome
4,9,Cattaraugus
...,...,...
57,115,Washington
58,117,Wayne
59,119,Westchester
60,121,Wyoming


In [ ]:
# Rename the categories so it's much more clear.
ny_county_fips_df = ny_county_fips_df.rename(columns={"p": "County_FIPS", "p 2": "County_Name"})

In [ ]:
ny_county_fips_df

,County_FIPS,County_Name
0,1,Albany
1,3,Allegany
2,5,Bronx
3,7,Broome
4,9,Cattaraugus
...,...,...
57,115,Washington
58,117,Wayne
59,119,Westchester
60,121,Wyoming


In [ ]:
# Just to make sure all the fips codes work, use string and zfill to make all fips county codes 3 digits.
ny_county_fips_df['County_FIPS'] = ny_county_fips_df['County_FIPS'].astype(str).str.zfill(3)

In [ ]:
ny_county_fips_df

,County_FIPS,County_Name
0,001,Albany
1,003,Allegany
2,005,Bronx
3,007,Broome
4,009,Cattaraugus
...,...,...
57,115,Washington
58,117,Wayne
59,119,Westchester
60,121,Wyoming


In [ ]:
# This hides the index column.
#print(ny_county_fips_df.to_string(index=False))

In [ ]:
# This drops the city regions from the list of counties.
nyc_counties = ["New York", "Kings", "Bronx", "Richmond", "Queens"]
applicable_fips_df = ny_county_fips_df
for x in nyc_counties:
  applicable_fips_df = applicable_fips_df.drop(applicable_fips_df[applicable_fips_df["County_Name"] == x].index)
applicable_fips_df

,County_FIPS,County_Name
0,001,Albany
1,003,Allegany
3,007,Broome
4,009,Cattaraugus
5,011,Cayuga
6,013,Chautauqua
7,015,Chemung
8,017,Chenango
9,019,Clinton
10,021,Columbia


## ACS Data (Izabela mostly, partly Dani)

Now returning to the ACS data, we will specify 5-year PUMS data.

**2023 ACS Data**

In [ ]:
# This gathers all of the data that we want to look into.
acs_cols = [
    "B01003_001E", # Total population
    "B25077_001E", # Median value of housing units in dollars (the value is the respondent's estimate of how much the property would sell for if it were for sale)

    "B19059_002E", # Population of people with retirement income in the past 12 months
    "B19059_003E", # Population of people without retirement income in the past 12 months

    "B02001_002E", # Population of White people
    "B02001_003E", # Population of Black/African American people
    "B02001_004E", # Population of American Indian/Alaska Native people
    "B02001_005E", # Population of Asian people
    "B02001_006E", # Population of Native Hawaiian/Other Pacific Islander people
    "B02001_007E", # Population of Other
    "B02001_008E", # Population of Two or More Races
    "B02001_009E", # Population of Two Races including Some Other Race
    "B02001_010E", # Population of Two Races excluding Some Other Race, and Three or More Races
    "B03003_003E", # Hispanic or Latino origin

    "B99233_003E", # Allocated usual hours worked per week in the past 12 months 16+
    "B99233_004E", # Non-Allocated usual hours worked per week in the past 12 months 16+

    "B15003_017E", # HS Diploma (Educational attainment for the population 25 years and over)
    "B15003_018E", # GED (Educational attainment for the population 25 years and over)
    "B15003_021E", # Associate's Degree (Educational attainment for the population 25 years and over)
    "B15003_022E", # Bachelor's Degree (Educational attainment for the population 25 years and over)
    "B15003_023E", # Master's Degree (Educational attainment for the population 25 years and over)
    "B15003_024E", # Professional School Degree (Educational attainment for the population 25 years and over)
    "B15003_025E", # Doctorate Degree (Educational attainment for the population 25 years and over)

    "C17002_002E", # Under 0.50 income-to-poverty ratio in the past 12 months
    "C17002_003E", # Between 0.50 to 0.99 income-to-poverty ratio in the past 12 months
    "C17002_004E", # Between 1.00 to 1.24 income-to-poverty ratio in the past 12 months
    "C17002_005E", # Between 1.25 to 1.49 income-to-poverty ratio in the past 12 months
    "C17002_006E", # Between 1.50 to 1.84 income-to-poverty ratio in the past 12 months
    "C17002_007E", # Between 1.85 to 1.99 income-to-poverty ratio in the past 12 months
    "C17002_008E", # 2.00 and over income-to-poverty ratio in the past 12 months
]
acs_colnames = [
    "TOTPOP", # Total population
    "HOMEMED", # Median value of housing units in dollars (the value is the respondent's estimate of how much the property would sell for if it were for sale)

    "RETINC", # Population of people with retirement income in the past 12 months
    "NORETINC", # Population of people without retirement income in the past 12 months

    "WHITE", # Population of White people
    "BLACK", # Population of Black/African American people
    "AMIN", # Population of American Indian/Alaska Native people
    "ASIAN", # Population of Asian people
    "NHPI", # Population of Native Hawaiian/Other Pacific Islander people
    "OTHER", # Population of Other
    "TWO_OR_MORE", # Population of Two or More Races
    "TWO_AND_SOME_OTHER", # Population of Two Races including Some Other Race
    "TWO_WO_SOMEOTHER_AND_THREEORMORE", # Population of Two Races excluding Some Other Race, and Three or More Races
    "HISP", # Hispanic or Latino origin

    "ALLWORK", # Allocated usual hours worked per week in the past 12 months 16+
    "NONALLWORK", # Non-Allocated usual hours worked per week in the past 12 months 16+

    "HS_SCHL", # HS Diploma (Educational attainment for the population 25 years and over)
    "GED_SCHL", # GED (Educational attainment for the population 25 years and over)
    "ASSOC_SCHL", # Associate's Degree (Educational attainment for the population 25 years and over)
    "BACH_SCHL", # Bachelor's Degree (Educational attainment for the population 25 years and over)
    "MAS_SCHL", # Master's Degree (Educational attainment for the population 25 years and over)
    "PROF_SCHL", # Professional School Degree (Educational attainment for the population 25 years and over)
    "DOC_SCHL", # Doctorate Degree (Educational attainment for the population 25 years and over)

    "POV_BEL_0.50", # Under 0.50 income-to-poverty ratio in the past 12 months
    "POV_BET_0.50_0.99", # Between 0.50 to 0.99 income-to-poverty ratio in the past 12 months
    "POV_BET_1.00_1.24", # Between 1.00 to 1.24 income-to-poverty ratio in the past 12 months
    "POV_BET_1.25_1.49", # Between 1.25 to 1.49 income-to-poverty ratio in the past 12 months
    "POV_BET_1.50_1.84", # Between 1.50 to 1.84 income-to-poverty ratio in the past 12 months
    "POV_BET_1.85_1.99", # Between 1.85 to 1.99 income-to-poverty ratio in the past 12 months
    "POV_ABV_2.00", # 2.00 and over income-to-poverty ratio in the past 12 months
]

In [ ]:
# Turns the FIPS codes of every non-city county into a list.
#county_fips = applicable_fips_df['County_FIPS'].tolist()

In [ ]:
acs_df = pd.DataFrame(
    census.acs5.get(
        tuple(acs_cols),
        geo={
            "for": "county:*",
            "in": "state:36",
        },
        year=2023,
    )
)

In [ ]:
acs23_df_clean = acs_df.rename(
    columns={x: y for x, y in zip(acs_cols, acs_colnames)}
)
acs23_df_clean["GEOID20"] = (
    acs23_df_clean["state"] + acs23_df_clean["county"]
).astype(int)
acs23_df_clean.set_index("GEOID20", inplace=True)
acs23_df_clean.head()

,TOTPOP,HOMEMED,RETINC,NORETINC,WHITE,BLACK,AMIN,ASIAN,NHPI,OTHER,...,DOC_SCHL,POV_BEL_0.50,POV_BET_0.50_0.99,POV_BET_1.00_1.24,POV_BET_1.25_1.49,POV_BET_1.50_1.84,POV_BET_1.85_1.99,POV_ABV_2.00,state,county
GEOID20,,,,,,,,,,,,,,,,,,,,,
36001,315374.0,277400.0,39109.0,93619.0,223239.0,38696.0,238.0,22969.0,187.0,8010.0,...,5987.0,21113.0,17587.0,9417.0,6841.0,11955.0,6202.0,226464.0,36,001
36003,47027.0,97900.0,5646.0,11411.0,44094.0,801.0,82.0,627.0,26.0,392.0,...,446.0,3717.0,3378.0,1720.0,2005.0,2549.0,1410.0,27454.0,36,003
36005,1419250.0,517000.0,88939.0,441128.0,223793.0,484151.0,16746.0,57790.0,2094.0,451233.0,...,6054.0,188999.0,185139.0,83139.0,77734.0,101883.0,37079.0,714441.0,36,005
36007,197738.0,145100.0,26193.0,55588.0,161027.0,10789.0,273.0,9873.0,92.0,3918.0,...,2357.0,18926.0,16522.0,7993.0,6899.0,10738.0,5016.0,121197.0,36,007
36009,76479.0,109400.0,9666.0,22156.0,68357.0,1125.0,2013.0,555.0,0.0,710.0,...,470.0,5673.0,7366.0,3709.0,3562.0,6125.0,2242.0,45417.0,36,009


In [ ]:
# Renames the "county" column to "County".
acs23_df_clean = acs23_df_clean.rename(columns={"county": "County"})

In [ ]:
applicable_fips_df = applicable_fips_df.reset_index()

In [ ]:
# This creates a dictionary with EVERY (city and non-city) FIPS code.
# The keys are the FIPS codes and the values are the city names.
FIPS_to_Names = {}
index = 0
for x in ny_county_fips_df["County_FIPS"]:
  FIPS_to_Names[x] = ny_county_fips_df.loc[index, "County_Name"]
  index += 1
print(FIPS_to_Names)

{'001': 'Albany', '003': 'Allegany', '005': 'Bronx', '007': 'Broome', '009': 'Cattaraugus', '011': 'Cayuga', '013': 'Chautauqua', '015': 'Chemung', '017': 'Chenango', '019': 'Clinton', '021': 'Columbia', '023': 'Cortland', '025': 'Delaware', '027': 'Dutchess', '029': 'Erie', '031': 'Essex', '033': 'Franklin', '035': 'Fulton', '037': 'Genesee', '039': 'Greene', '041': 'Hamilton', '043': 'Herkimer', '045': 'Jefferson', '047': 'Kings', '049': 'Lewis', '051': 'Livingston', '053': 'Madison', '055': 'Monroe', '057': 'Montgomery', '059': 'Nassau', '061': 'New York', '063': 'Niagara', '065': 'Oneida', '067': 'Onondaga', '069': 'Ontario', '071': 'Orange', '073': 'Orleans', '075': 'Oswego', '077': 'Otsego', '079': 'Putnam', '081': 'Queens', '083': 'Rensselaer', '085': 'Richmond', '087': 'Rockland', '089': 'St. Lawrence', '091': 'Saratoga', '093': 'Schenectady', '095': 'Schoharie', '097': 'Schuyler', '099': 'Seneca', '101': 'Steuben', '103': 'Suffolk', '105': 'Sullivan', '107': 'Tioga', '109': 'T

In [ ]:
# This renames all of the FIPS codes to county names in the ACS data.
for x in acs23_df_clean["County"]:
  for y in FIPS_to_Names:
    if x == y:
      acs23_df_clean["County"] = acs23_df_clean["County"].replace(x, FIPS_to_Names[y])
acs23_df_clean

,TOTPOP,HOMEMED,RETINC,NORETINC,WHITE,BLACK,AMIN,ASIAN,NHPI,OTHER,...,DOC_SCHL,POV_BEL_0.50,POV_BET_0.50_0.99,POV_BET_1.00_1.24,POV_BET_1.25_1.49,POV_BET_1.50_1.84,POV_BET_1.85_1.99,POV_ABV_2.00,state,County
GEOID20,,,,,,,,,,,,,,,,,,,,,
36001,315374.0,277400.0,39109.0,93619.0,223239.0,38696.0,238.0,22969.0,187.0,8010.0,...,5987.0,21113.0,17587.0,9417.0,6841.0,11955.0,6202.0,226464.0,36,Albany
36003,47027.0,97900.0,5646.0,11411.0,44094.0,801.0,82.0,627.0,26.0,392.0,...,446.0,3717.0,3378.0,1720.0,2005.0,2549.0,1410.0,27454.0,36,Allegany
36005,1419250.0,517000.0,88939.0,441128.0,223793.0,484151.0,16746.0,57790.0,2094.0,451233.0,...,6054.0,188999.0,185139.0,83139.0,77734.0,101883.0,37079.0,714441.0,36,Bronx
36007,197738.0,145100.0,26193.0,55588.0,161027.0,10789.0,273.0,9873.0,92.0,3918.0,...,2357.0,18926.0,16522.0,7993.0,6899.0,10738.0,5016.0,121197.0,36,Broome
36009,76479.0,109400.0,9666.0,22156.0,68357.0,1125.0,2013.0,555.0,0.0,710.0,...,470.0,5673.0,7366.0,3709.0,3562.0,6125.0,2242.0,45417.0,36,Cattaraugus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36115,60883.0,185000.0,7325.0,16929.0,55130.0,1504.0,44.0,376.0,20.0,1224.0,...,316.0,2465.0,3839.0,1410.0,2882.0,3302.0,2096.0,42160.0,36,Washington
36117,91128.0,163400.0,12095.0,26127.0,80916.0,2164.0,285.0,788.0,37.0,1811.0,...,349.0,5057.0,5117.0,3853.0,2781.0,4678.0,2655.0,66133.0,36,Wayne
36119,996888.0,638400.0,94894.0,275362.0,545304.0,139205.0,5191.0,61001.0,158.0,133034.0,...,17361.0,45029.0,41641.0,22235.0,26599.0,35057.0,15953.0,789593.0,36,Westchester


In [ ]:
# Ensures that we aren't using any city counties. (Gets rid of them from the ACS data).
nyc_counties = ["New York", "Kings", "Bronx", "Richmond", "Queens"]
for x in nyc_counties:
  acs23_df_clean = acs23_df_clean.drop(acs23_df_clean[acs23_df_clean["County"] == x].index)

Dani here, doing some calculations so it's easier to interpret acs data in it's respective contexts.

In [ ]:
acs23_df_clean['Percent of People with Retirement Income'] = (acs23_df_clean['RETINC'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People without Retirement Income'] = (acs23_df_clean['NORETINC'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of White People'] = (acs23_df_clean['WHITE'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of Black People'] = (acs23_df_clean['BLACK'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of American Indian People'] = (acs23_df_clean['AMIN'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of Asian People'] = (acs23_df_clean['ASIAN'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of Native Hawaiian People'] = (acs23_df_clean['NHPI'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of Other People (Race)'] = (acs23_df_clean['OTHER'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People with Two or More Races'] = (acs23_df_clean['TWO_OR_MORE'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People with Two and Some Other Races'] = (acs23_df_clean['TWO_AND_SOME_OTHER'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People with Two Races Excluding Some Other Race and Three or More Races'] = (acs23_df_clean['TWO_WO_SOMEOTHER_AND_THREEORMORE'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of Hispanic People'] = (acs23_df_clean['HISP'] / acs23_df_clean['TOTPOP']) * 100

acs23_df_clean['Percent of People whose Highest Degree is a High School Diploma'] = (acs23_df_clean['HS_SCHL'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People whose Highest Degree a GED'] = (acs23_df_clean['GED_SCHL'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People whose Highest Degree is an Associate\'s Degree'] = (acs23_df_clean['ASSOC_SCHL'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People whose Highest Degree is a Bachelor\'s Degree'] = (acs23_df_clean['BACH_SCHL'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People whose Highest Degree is a Master\'s Degree'] = (acs23_df_clean['MAS_SCHL'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People whose Highest Degree is a Professional School Degree'] = (acs23_df_clean['PROF_SCHL'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People whose Highest Degree is a Doctorate Degree'] = (acs23_df_clean['DOC_SCHL'] / acs23_df_clean['TOTPOP']) * 100

acs23_df_clean['Percent of People at 50% below Federal Poverty Line'] = (acs23_df_clean['POV_BEL_0.50'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People between 50% and 99% below Federal Poverty Line'] = (acs23_df_clean['POV_BET_0.50_0.99'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People between 100% (at Federal Poverty Line) and 124% (above Federal Poverty Line)'] = (acs23_df_clean['POV_BET_1.00_1.24'] / acs23_df_clean['TOTPOP'])* 100
acs23_df_clean['Percent of People between 125% and 149% above Federal Poverty Line'] = (acs23_df_clean['POV_BET_1.25_1.49'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People between 150% and 184% above Federal Poverty Line'] = (acs23_df_clean['POV_BET_1.50_1.84'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People between 185% and 199% above Federal Poverty Line'] = (acs23_df_clean['POV_BET_1.85_1.99'] / acs23_df_clean['TOTPOP']) * 100
acs23_df_clean['Percent of People at 200% or above Federal Poverty Line'] = (acs23_df_clean['POV_ABV_2.00'] / acs23_df_clean['TOTPOP']) * 100


That's it for my part, I'm going to repeat this in the 2019 part as well.

In [ ]:
# Merges the dispensary data and the ACS data together.
disp_acs23_mgd_gdf = pd.merge(final_disp_gdf, acs23_df_clean)
disp_acs23_mgd_gdf

,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,SEE Category,Entity Name,...,Percent of People whose Highest Degree is a Master's Degree,Percent of People whose Highest Degree is a Professional School Degree,Percent of People whose Highest Degree is a Doctorate Degree,Percent of People at 50% below Federal Poverty Line,Percent of People between 50% and 99% below Federal Poverty Line,Percent of People between 100% (at Federal Poverty Line) and 124% (above Federal Poverty Line),Percent of People between 125% and 149% above Federal Poverty Line,Percent of People between 150% and 184% above Federal Poverty Line,Percent of People between 185% and 199% above Federal Poverty Line,Percent of People at 200% or above Federal Poverty Line
0,OCM-RETL-24-000123,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/10/2024 12:00:00 AM,07/10/2024 12:00:00 AM,07/10/2026 12:00:00 AM,NaN,10415 Jamaica LLC,...,7.137233,1.074931,0.804279,6.142284,6.545414,3.792591,3.124010,4.421553,2.063936,71.820771
1,OCM-RETL-24-000138,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/16/2024 12:00:00 AM,07/16/2024 12:00:00 AM,07/16/2026 12:00:00 AM,NaN,1650 CRANE STREET DISPENSARY LLC,...,7.925916,1.608992,1.280051,6.433463,5.711672,3.324499,2.891550,4.837628,1.890315,72.314430
2,OCM-RETL-24-000081,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,NaN,1WILDE1 LLC,...,8.623496,1.371452,1.169383,4.766092,6.570372,2.809013,2.900693,3.944718,2.169751,73.042453
3,OCM-RETL-24-000267,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,12/19/2024 12:00:00 AM,12/19/2024 12:00:00 AM,12/19/2026 12:00:00 AM,NaN,296 Retail Venture LLC,...,8.408077,2.213987,1.018481,6.995421,8.337562,3.657328,3.582977,4.445382,2.166191,69.386846
4,OCM-RETL-24-000082,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,"Minority-Owned Business, Women-Owned Business",3807 harlem Cannabis LLC,...,8.618864,1.706553,1.327412,6.683269,7.001255,3.904135,3.612885,4.782936,2.232528,69.365314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,OCM-CAURD-24-000138,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,05/03/2024 12:00:00 AM,05/03/2024 12:00:00 AM,05/03/2026 12:00:00 AM,NaN,Valley Greens Ltd.,...,12.588977,4.189036,1.741520,4.516957,4.177099,2.230441,2.668203,3.516644,1.600280,79.205788
436,OCM-CAURD-24-000198,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,08/23/2024 12:00:00 AM,08/23/2024 12:00:00 AM,08/23/2026 12:00:00 AM,NaN,Vedaleaf LLC,...,8.727320,1.787066,1.205867,7.118367,6.215504,3.543386,3.935659,4.506892,2.336884,68.065630
437,OCM-CAURD-24-000124,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,04/12/2024 12:00:00 AM,04/12/2024 12:00:00 AM,04/12/2026 12:00:00 AM,NaN,Westchester Harvesting Company LLC,...,12.588977,4.189036,1.741520,4.516957,4.177099,2.230441,2.668203,3.516644,1.600280,79.205788
438,OCM-CAURD-23-000006,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,03/10/2023 12:00:00 AM,03/10/2023 12:00:00 AM,03/10/2025 12:00:00 AM,NaN,WilliamJane Corporation,...,11.409520,2.026653,6.949912,9.163192,4.004705,2.469892,2.448507,4.887295,2.107330,61.034808


In [ ]:
# The merged dispensary, HRI, and ACS data.
hri_disp_acs23_mgd_gdf = pd.merge(joined_gdf_dropped, acs23_df_clean)

**2019 ACS Data**

In [ ]:
#apply these variables and run  # Changed the variable name to 'B17020_002E', which represents the 'Income In The Past 12 Months Below Poverty Level' field in the 2019 ACS 5-year data.


In [ ]:
acs_df = pd.DataFrame(
    census.acs5.get(
        tuple(acs_cols),
        geo={
            "for": "county:*",
            "in": "state:36",
        },
        year=2019,
    )
)

In [ ]:
acs19_df_clean = acs_df.rename(
    columns={x: y for x, y in zip(acs_cols, acs_colnames)}
)
acs19_df_clean["GEOID10"] = (
    acs19_df_clean["state"] + acs19_df_clean["county"]
).astype(int)
acs19_df_clean.set_index("GEOID10", inplace=True)
acs19_df_clean.head()

,TOTPOP,HOMEMED,RETINC,NORETINC,WHITE,BLACK,AMIN,ASIAN,NHPI,OTHER,...,DOC_SCHL,POV_BEL_0.50,POV_BET_0.50_0.99,POV_BET_1.00_1.24,POV_BET_1.25_1.49,POV_BET_1.50_1.84,POV_BET_1.85_1.99,POV_ABV_2.00,state,county
GEOID10,,,,,,,,,,,,,,,,,,,,,
36095,31222.0,144800.0,3642.0,8917.0,29791.0,496.0,48.0,281.0,9.0,170.0,...,171.0,1754.0,1858.0,1671.0,980.0,1722.0,488.0,21369.0,36,095
36035,53646.0,109600.0,5181.0,17376.0,50640.0,1119.0,209.0,417.0,7.0,444.0,...,114.0,3368.0,4446.0,2935.0,2338.0,4763.0,1505.0,33080.0,36,035
36083,159185.0,188700.0,16698.0,48208.0,137002.0,10268.0,228.0,4282.0,13.0,1675.0,...,1506.0,8885.0,9078.0,5142.0,4441.0,7341.0,3241.0,115287.0,36,083
36033,50477.0,108700.0,5503.0,13512.0,41726.0,2863.0,3609.0,280.0,111.0,1057.0,...,187.0,3948.0,4033.0,2340.0,1775.0,3675.0,1244.0,27875.0,36,033
36081,2287388.0,543800.0,124749.0,654183.0,875191.0,418429.0,10326.0,583459.0,1078.0,316141.0,...,16075.0,113970.0,162477.0,104435.0,104862.0,154683.0,60151.0,1559361.0,36,081


In [ ]:
# Renames the "county" column to "County".
acs19_df_clean = acs19_df_clean.rename(columns={"county": "County"})

In [ ]:
# This renames all of the FIPS codes to county names in the ACS data.
for x in acs19_df_clean["County"]:
  for y in FIPS_to_Names:
    if x == y:
      acs19_df_clean["County"] = acs19_df_clean["County"].replace(x, FIPS_to_Names[y])
acs19_df_clean

,TOTPOP,HOMEMED,RETINC,NORETINC,WHITE,BLACK,AMIN,ASIAN,NHPI,OTHER,...,DOC_SCHL,POV_BEL_0.50,POV_BET_0.50_0.99,POV_BET_1.00_1.24,POV_BET_1.25_1.49,POV_BET_1.50_1.84,POV_BET_1.85_1.99,POV_ABV_2.00,state,County
GEOID10,,,,,,,,,,,,,,,,,,,,,
36095,31222.0,144800.0,3642.0,8917.0,29791.0,496.0,48.0,281.0,9.0,170.0,...,171.0,1754.0,1858.0,1671.0,980.0,1722.0,488.0,21369.0,36,Schoharie
36035,53646.0,109600.0,5181.0,17376.0,50640.0,1119.0,209.0,417.0,7.0,444.0,...,114.0,3368.0,4446.0,2935.0,2338.0,4763.0,1505.0,33080.0,36,Fulton
36083,159185.0,188700.0,16698.0,48208.0,137002.0,10268.0,228.0,4282.0,13.0,1675.0,...,1506.0,8885.0,9078.0,5142.0,4441.0,7341.0,3241.0,115287.0,36,Rensselaer
36033,50477.0,108700.0,5503.0,13512.0,41726.0,2863.0,3609.0,280.0,111.0,1057.0,...,187.0,3948.0,4033.0,2340.0,1775.0,3675.0,1244.0,27875.0,36,Franklin
36081,2287388.0,543800.0,124749.0,654183.0,875191.0,418429.0,10326.0,583459.0,1078.0,316141.0,...,16075.0,113970.0,162477.0,104435.0,104862.0,154683.0,60151.0,1559361.0,36,Queens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36009,77121.0,88100.0,8256.0,23523.0,70859.0,1116.0,2485.0,610.0,0.0,470.0,...,329.0,4960.0,7364.0,4653.0,4043.0,5450.0,2376.0,45833.0,36,Cattaraugus
36053,71205.0,136800.0,6577.0,19300.0,67360.0,1268.0,339.0,638.0,79.0,309.0,...,586.0,3089.0,3370.0,2655.0,2990.0,3390.0,2171.0,48392.0,36,Madison
36093,154859.0,169600.0,14009.0,40293.0,118130.0,15590.0,359.0,7228.0,84.0,6079.0,...,2080.0,7233.0,10038.0,5597.0,4944.0,8566.0,2407.0,112062.0,36,Schenectady


In [ ]:
# Ensures that we aren't using any city counties. (Gets rid of them from the ACS data).
nyc_counties = ["New York", "Kings", "Bronx", "Richmond", "Queens"]
for x in nyc_counties:
  acs19_df_clean = acs19_df_clean.drop(acs19_df_clean[acs19_df_clean["County"] == x].index)

In [ ]:
acs19_df_clean['Percent of People with Retirement Income'] = (acs19_df_clean['RETINC'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People without Retirement Income'] = (acs19_df_clean['NORETINC'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of White People'] = (acs19_df_clean['WHITE'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of Black People'] = (acs19_df_clean['BLACK'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of American Indian People'] = (acs19_df_clean['AMIN'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of Asian People'] = (acs19_df_clean['ASIAN'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of Native Hawaiian People'] = (acs19_df_clean['NHPI'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of Other People (Race)'] = (acs19_df_clean['OTHER'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of Two or More Races'] = (acs19_df_clean['TWO_OR_MORE'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People with Two and Some Other Races'] = (acs19_df_clean['TWO_AND_SOME_OTHER'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People with Two Races Excluding Some Other Race and Three or More Races'] = (acs19_df_clean['TWO_WO_SOMEOTHER_AND_THREEORMORE'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of Hispanic People'] = (acs19_df_clean['HISP'] / acs19_df_clean['TOTPOP']) * 100

acs19_df_clean['Percent of People whose Highest Degree is a High School Diploma'] = (acs19_df_clean['HS_SCHL'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People whose Highest Degree a GED'] = (acs19_df_clean['GED_SCHL'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People whose Highest Degree is an Associate\'s Degree'] = (acs19_df_clean['ASSOC_SCHL'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People whose Highest Degree is a Bachelor\'s Degree'] = (acs19_df_clean['BACH_SCHL'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People whose Highest Degree is a Master\'s Degree'] = (acs19_df_clean['MAS_SCHL'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People whose Highest Degree is a Professional School Degree'] = (acs19_df_clean['PROF_SCHL'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People whose Highest Degree is a Doctorate Degree'] = (acs19_df_clean['DOC_SCHL'] / acs19_df_clean['TOTPOP']) * 100

acs19_df_clean['Percent of People at 50% below Federal Poverty Line'] = (acs19_df_clean['POV_BEL_0.50'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People between 50% and 99% below Federal Poverty Line'] = (acs19_df_clean['POV_BET_0.50_0.99'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People between 100% (at Federal Poverty Line) and 124% (above Federal Poverty Line)'] = (acs19_df_clean['POV_BET_1.00_1.24'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People between 125% and 149% above Federal Poverty Line'] = (acs19_df_clean['POV_BET_1.25_1.49'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People between 150% and 184% above Federal Poverty Line'] = (acs19_df_clean['POV_BET_1.50_1.84'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People between 185% and 199% above Federal Poverty Line'] = (acs19_df_clean['POV_BET_1.85_1.99'] / acs19_df_clean['TOTPOP']) * 100
acs19_df_clean['Percent of People at 200% or above Federal Poverty Line'] = (acs19_df_clean['POV_ABV_2.00'] / acs19_df_clean['TOTPOP']) * 100


In [ ]:
# Merges the dispensary data and the ACS data together.
disp_acs19_mgd_gdf = pd.merge(final_disp_gdf, acs19_df_clean)
disp_acs19_mgd_gdf

,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,SEE Category,Entity Name,...,Percent of People whose Highest Degree is a Master's Degree,Percent of People whose Highest Degree is a Professional School Degree,Percent of People whose Highest Degree is a Doctorate Degree,Percent of People at 50% below Federal Poverty Line,Percent of People between 50% and 99% below Federal Poverty Line,Percent of People between 100% (at Federal Poverty Line) and 124% (above Federal Poverty Line),Percent of People between 125% and 149% above Federal Poverty Line,Percent of People between 150% and 184% above Federal Poverty Line,Percent of People between 185% and 199% above Federal Poverty Line,Percent of People at 200% or above Federal Poverty Line
0,OCM-RETL-24-000123,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/10/2024 12:00:00 AM,07/10/2024 12:00:00 AM,07/10/2026 12:00:00 AM,NaN,10415 Jamaica LLC,...,6.606680,1.001355,0.638541,5.194891,6.155728,3.435284,3.257429,4.921794,2.131628,72.339871
1,OCM-RETL-24-000138,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/16/2024 12:00:00 AM,07/16/2024 12:00:00 AM,07/16/2026 12:00:00 AM,NaN,1650 CRANE STREET DISPENSARY LLC,...,6.908865,1.122957,1.343157,4.670700,6.482026,3.614256,3.192582,5.531483,1.554317,72.363892
2,OCM-RETL-24-000081,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,NaN,1WILDE1 LLC,...,7.512014,1.376386,0.946069,5.581556,5.702799,3.230204,2.789836,4.611615,2.035996,72.423281
3,OCM-RETL-24-000267,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,12/19/2024 12:00:00 AM,12/19/2024 12:00:00 AM,12/19/2026 12:00:00 AM,NaN,296 Retail Venture LLC,...,8.623336,1.997398,1.094254,6.262522,7.424589,3.524730,3.726011,5.267522,1.682377,70.789897
4,OCM-RETL-24-000082,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,"Minority-Owned Business, Women-Owned Business",3807 harlem Cannabis LLC,...,7.830490,1.593617,1.075972,6.701655,7.091276,3.851722,3.518445,5.261080,2.190775,68.521518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,OCM-CAURD-24-000138,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,05/03/2024 12:00:00 AM,05/03/2024 12:00:00 AM,05/03/2026 12:00:00 AM,NaN,Valley Greens Ltd.,...,11.608955,3.894560,1.519780,4.187266,4.441474,3.169503,2.707222,3.745936,1.499241,77.860541
436,OCM-CAURD-24-000198,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,08/23/2024 12:00:00 AM,08/23/2024 12:00:00 AM,08/23/2026 12:00:00 AM,NaN,Vedaleaf LLC,...,7.856168,1.839169,1.156907,6.643521,6.882896,3.404397,3.786360,4.763088,2.283569,68.087074
437,OCM-CAURD-24-000124,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,04/12/2024 12:00:00 AM,04/12/2024 12:00:00 AM,04/12/2026 12:00:00 AM,NaN,Westchester Harvesting Company LLC,...,11.608955,3.894560,1.519780,4.187266,4.441474,3.169503,2.707222,3.745936,1.499241,77.860541
438,OCM-CAURD-23-000006,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,03/10/2023 12:00:00 AM,03/10/2023 12:00:00 AM,03/10/2025 12:00:00 AM,NaN,WilliamJane Corporation,...,10.575593,1.826738,5.550360,10.525906,5.878685,2.595429,2.200853,4.447497,2.649013,58.528672


In [ ]:
# The merged dispensary, HRI, and ACS data.
hri_disp_acs19_mgd_gdf = pd.merge(joined_gdf_dropped, acs19_df_clean)

# Cannabis/Substance Arrest Data by County (Dani)

https://data.ny.gov/Public-Safety/Adult-Arrests-18-and-Older-by-County-Beginning-197/rikd-mt35/about_data

This was a last minute addition, so some filtering was done within the NY Open Data Platform before downloading as a CSV. We're looking at 2 main categories (this is the closest we could find to cannabis arrests by county, note that this category does include ALL controlled substances as well).

1. Drug misdemeanor, which looks at
Number of adult arrests in which the top charge is a Penal Law §220 (Controlled Substances) or §221 (Marijuana) misdemeanor.
2. Drug felony, which looks at
Number of adult arrests in which the top charge is a felony listed in Penal Law §220 (Controlled Substances), §221 (Marijuana), or §221 (Cannabis)

We filtered out non-NYC counties as well, keeping county, year, total of all arrests, total felonies, then total for each categories. The only difference between the 2 csvs are that one is filtered to 2019 data and the other is filtered to 2023 data.

Filter by year and removing non NYC counties was done before downloading data.


In [ ]:
arrests_2019df = pd.read_csv("/content/2019nonnycdrugarrests.csv")

In [ ]:
#arrests_2019df

In [ ]:
# Filtering down to the desired categories.
arrests_2019df = arrests_2019df[['County', 'Year', 'Total', 'Felony Total', 'Drug Felony', 'Drug Misdemeanor']]
arrests_2019df

,County,Year,Total,Felony Total,Drug Felony,Drug Misdemeanor
0,Albany,2019,6232,2288,418,674
1,Allegany,2019,752,210,33,60
2,Broome,2019,4807,1425,293,659
3,Cattaraugus,2019,1523,453,129,163
4,Cayuga,2019,1241,411,66,62
5,Chautauqua,2019,3603,1081,207,463
6,Chemung,2019,2039,525,103,358
7,Chenango,2019,787,250,38,83
8,Clinton,2019,1708,460,111,116
9,Columbia,2019,1263,340,85,329


In [ ]:
arrests_2019df = arrests_2019df.rename(columns={"Total": "Total Adults Arrested"})

In [ ]:
#arrests_2019df

In [ ]:
# Calculating total drug offenses.
arrests_2019df['Total Drug Offenses'] = arrests_2019df['Drug Felony'] + arrests_2019df['Drug Misdemeanor']
#arrests_2019df

In [ ]:
# Creating new columms for new percentage calculations.
arrests_2019df['Percent of Drug Felony in Overall Felony'] = (arrests_2019df['Drug Felony'] / arrests_2019df['Felony Total']) * 100
arrests_2019df['Percent of Drug Misdemeanor in Overall Adult Arrests'] = (arrests_2019df['Drug Misdemeanor'] / arrests_2019df['Total Adults Arrested']) * 100
arrests_2019df['Percent of Drug Felony in Overall Adult Arrests'] = (arrests_2019df['Drug Felony'] / arrests_2019df['Total Adults Arrested']) * 100
arrests_2019df['Percent of Overall Drug Offenses in Overall Adult Arrests'] = (arrests_2019df['Total Drug Offenses'] / arrests_2019df['Total Adults Arrested']) * 100


In [ ]:
# Now we have our 2019 dataframe with percentage calculations.
arrests_2019df

,County,Year,Total Adults Arrested,Felony Total,Drug Felony,Drug Misdemeanor,Total Drug Offenses,Percent of Drug Felony in Overall Felony,Percent of Drug Misdemeanor in Overall Adult Arrests,Percent of Drug Felony in Overall Adult Arrests,Percent of Overall Drug Offenses in Overall Adult Arrests
0,Albany,2019,6232,2288,418,674,1092,18.269231,10.815148,6.707317,17.522465
1,Allegany,2019,752,210,33,60,93,15.714286,7.978723,4.388298,12.367021
2,Broome,2019,4807,1425,293,659,952,20.561404,13.709174,6.095278,19.804452
3,Cattaraugus,2019,1523,453,129,163,292,28.476821,10.702561,8.470125,19.172685
4,Cayuga,2019,1241,411,66,62,128,16.058394,4.995971,5.318292,10.314263
5,Chautauqua,2019,3603,1081,207,463,670,19.148936,12.850402,5.745212,18.595615
6,Chemung,2019,2039,525,103,358,461,19.619048,17.557626,5.051496,22.609122
7,Chenango,2019,787,250,38,83,121,15.200000,10.546379,4.828463,15.374841
8,Clinton,2019,1708,460,111,116,227,24.130435,6.791569,6.498829,13.290398
9,Columbia,2019,1263,340,85,329,414,25.000000,26.049089,6.730008,32.779097


In [ ]:
# Merge with acs, hci, and dispo data, 2019.

all_data_2019 = pd.merge(disp_acs19_mgd_gdf, arrests_2019df)
all_data_2019

,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,SEE Category,Entity Name,...,Year,Total Adults Arrested,Felony Total,Drug Felony,Drug Misdemeanor,Total Drug Offenses,Percent of Drug Felony in Overall Felony,Percent of Drug Misdemeanor in Overall Adult Arrests,Percent of Drug Felony in Overall Adult Arrests,Percent of Overall Drug Offenses in Overall Adult Arrests
0,OCM-RETL-24-000123,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/10/2024 12:00:00 AM,07/10/2024 12:00:00 AM,07/10/2026 12:00:00 AM,NaN,10415 Jamaica LLC,...,2019,7835,2183,480,1125,1605,21.988090,14.358647,6.126356,20.485003
1,OCM-RETL-24-000138,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/16/2024 12:00:00 AM,07/16/2024 12:00:00 AM,07/16/2026 12:00:00 AM,NaN,1650 CRANE STREET DISPENSARY LLC,...,2019,3407,1200,242,201,443,20.166667,5.899618,7.103023,13.002642
2,OCM-RETL-24-000081,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,NaN,1WILDE1 LLC,...,2019,3254,1115,192,298,490,17.219731,9.157959,5.900430,15.058390
3,OCM-RETL-24-000267,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,12/19/2024 12:00:00 AM,12/19/2024 12:00:00 AM,12/19/2026 12:00:00 AM,NaN,296 Retail Venture LLC,...,2019,3085,930,144,320,464,15.483871,10.372771,4.667747,15.040519
4,OCM-RETL-24-000082,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,"Minority-Owned Business, Women-Owned Business",3807 harlem Cannabis LLC,...,2019,19181,6358,1283,2799,4082,20.179302,14.592566,6.688911,21.281476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,OCM-CAURD-24-000138,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,05/03/2024 12:00:00 AM,05/03/2024 12:00:00 AM,05/03/2026 12:00:00 AM,NaN,Valley Greens Ltd.,...,2019,12962,4189,712,1555,2267,16.996897,11.996605,5.492979,17.489585
436,OCM-CAURD-24-000198,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,08/23/2024 12:00:00 AM,08/23/2024 12:00:00 AM,08/23/2026 12:00:00 AM,NaN,Vedaleaf LLC,...,2019,9578,3217,415,834,1249,12.900218,8.707455,4.332846,13.040301
437,OCM-CAURD-24-000124,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,04/12/2024 12:00:00 AM,04/12/2024 12:00:00 AM,04/12/2026 12:00:00 AM,NaN,Westchester Harvesting Company LLC,...,2019,12962,4189,712,1555,2267,16.996897,11.996605,5.492979,17.489585
438,OCM-CAURD-23-000006,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,03/10/2023 12:00:00 AM,03/10/2023 12:00:00 AM,03/10/2025 12:00:00 AM,NaN,WilliamJane Corporation,...,2019,1108,267,30,91,121,11.235955,8.212996,2.707581,10.920578


In [ ]:
# Now repeat with 2023 data.
arrests_2023df = pd.read_csv("/content/2023nonnycdrugarrests.csv")
#arrests_2023df

In [ ]:
# Filtering down to the desired categories.
arrests_2023df = arrests_2023df[['County', 'Year', 'Total', 'Felony Total', 'Drug Felony', 'Drug Misdemeanor']]

# Calculating total drug offenses.
arrests_2023df['Total Drug Offenses'] = arrests_2023df['Drug Felony'] + arrests_2023df['Drug Misdemeanor']
#arrests_2023df

In [ ]:
# Renaming this column.
arrests_2023df = arrests_2023df.rename(columns={"Total": "Total Adults Arrested"})
arrests_2023df['Total Drug Offenses'] = arrests_2023df['Drug Felony'] + arrests_2023df['Drug Misdemeanor']


In [ ]:
arrests_2023df['Percent of Drug Felony in Overall Felony'] = (arrests_2023df['Drug Felony'] / arrests_2023df['Felony Total']) * 100
arrests_2023df['Percent of Drug Misdemeanor in Overall Adult Arrests'] = (arrests_2023df['Drug Misdemeanor'] / arrests_2023df['Total Adults Arrested']) * 100
arrests_2023df['Percent of Drug Felony in Overall Adult Arrests'] = (arrests_2023df['Drug Felony'] / arrests_2023df['Total Adults Arrested']) * 100
arrests_2023df['Percent of Overall Drug Offenses in Overall Adult Arrests'] = (arrests_2023df['Total Drug Offenses'] / arrests_2023df['Total Adults Arrested']) * 100


In [ ]:
# Now we have our 2023 arrests dataframe with percentages calculated.
arrests_2023df

,County,Year,Total Adults Arrested,Felony Total,Drug Felony,Drug Misdemeanor,Total Drug Offenses,Percent of Drug Felony in Overall Felony,Percent of Drug Misdemeanor in Overall Adult Arrests,Percent of Drug Felony in Overall Adult Arrests,Percent of Overall Drug Offenses in Overall Adult Arrests
0,Albany,2023,5597,2374,248,222,470,10.446504,3.966411,4.430945,8.397356
1,Allegany,2023,846,271,25,74,99,9.225092,8.747045,2.955083,11.702128
2,Broome,2023,4730,1549,207,489,696,13.363460,10.338266,4.376321,14.714588
3,Cattaraugus,2023,1789,610,110,175,285,18.032787,9.782001,6.148686,15.930688
4,Cayuga,2023,1209,425,37,43,80,8.705882,3.556658,3.060380,6.617039
5,Chautauqua,2023,3681,1243,228,424,652,18.342719,11.518609,6.193969,17.712578
6,Chemung,2023,1962,599,66,251,317,11.018364,12.793068,3.363914,16.156983
7,Chenango,2023,780,251,33,79,112,13.147410,10.128205,4.230769,14.358974
8,Clinton,2023,1413,432,70,187,257,16.203704,13.234253,4.953999,18.188252
9,Columbia,2023,1014,305,43,66,109,14.098361,6.508876,4.240631,10.749507


In [ ]:
# Merge with 2023 acs, dispo, and hci data.
all_data_2023 = pd.merge(disp_acs23_mgd_gdf, arrests_2023df)
all_data_2023

,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,SEE Category,Entity Name,...,Year,Total Adults Arrested,Felony Total,Drug Felony,Drug Misdemeanor,Total Drug Offenses,Percent of Drug Felony in Overall Felony,Percent of Drug Misdemeanor in Overall Adult Arrests,Percent of Drug Felony in Overall Adult Arrests,Percent of Overall Drug Offenses in Overall Adult Arrests
0,OCM-RETL-24-000123,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/10/2024 12:00:00 AM,07/10/2024 12:00:00 AM,07/10/2026 12:00:00 AM,NaN,10415 Jamaica LLC,...,2023,6479,1961,254,468,722,12.952575,7.223337,3.920358,11.143695
1,OCM-RETL-24-000138,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/16/2024 12:00:00 AM,07/16/2024 12:00:00 AM,07/16/2026 12:00:00 AM,NaN,1650 CRANE STREET DISPENSARY LLC,...,2023,3135,1080,60,78,138,5.555556,2.488038,1.913876,4.401914
2,OCM-RETL-24-000081,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,NaN,1WILDE1 LLC,...,2023,2669,989,108,112,220,10.920121,4.196328,4.046459,8.242788
3,OCM-RETL-24-000267,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,12/19/2024 12:00:00 AM,12/19/2024 12:00:00 AM,12/19/2026 12:00:00 AM,NaN,296 Retail Venture LLC,...,2023,3073,1000,57,130,187,5.700000,4.230394,1.854865,6.085259
4,OCM-RETL-24-000082,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,"Minority-Owned Business, Women-Owned Business",3807 harlem Cannabis LLC,...,2023,14290,5393,691,974,1665,12.812906,6.815955,4.835549,11.651505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,OCM-CAURD-24-000138,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,05/03/2024 12:00:00 AM,05/03/2024 12:00:00 AM,05/03/2026 12:00:00 AM,NaN,Valley Greens Ltd.,...,2023,11965,3995,335,668,1003,8.385482,5.582950,2.799833,8.382783
436,OCM-CAURD-24-000198,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,08/23/2024 12:00:00 AM,08/23/2024 12:00:00 AM,08/23/2026 12:00:00 AM,NaN,Vedaleaf LLC,...,2023,9352,3828,274,490,764,7.157785,5.239521,2.929855,8.169376
437,OCM-CAURD-24-000124,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,04/12/2024 12:00:00 AM,04/12/2024 12:00:00 AM,04/12/2026 12:00:00 AM,NaN,Westchester Harvesting Company LLC,...,2023,11965,3995,335,668,1003,8.385482,5.582950,2.799833,8.382783
438,OCM-CAURD-23-000006,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,03/10/2023 12:00:00 AM,03/10/2023 12:00:00 AM,03/10/2025 12:00:00 AM,NaN,WilliamJane Corporation,...,2023,1137,397,29,72,101,7.304786,6.332454,2.550572,8.883026


Time to make calculations.

In [ ]:
import pandas as pd
from pprint import pprint # Imports the pprint function.
# I want to calculate percent differences between 2023 and 2019, based on the percent calculations I already did.
# The code was being annoying, so I'm putting everything in again clearly so I can calculate overall percent differences within the percent rates I previously calculated.

arrests_2019df = pd.read_csv("/content/2019nonnycdrugarrests.csv")
arrests_2019df = arrests_2019df[['County', 'Year', 'Total', 'Felony Total', 'Drug Felony', 'Drug Misdemeanor']]
arrests_2019df = arrests_2019df.rename(columns={"Total": "Total Adults Arrested"})
arrests_2019df['Total Drug Offenses'] = arrests_2019df['Drug Felony'] + arrests_2019df['Drug Misdemeanor']
arrests_2019df['Percent of Drug Felony in Overall Felony'] = (arrests_2019df['Drug Felony'] / arrests_2019df['Felony Total']) * 100
arrests_2019df['Percent of Drug Misdemeanor in Overall Adult Arrests'] = (arrests_2019df['Drug Misdemeanor'] / arrests_2019df['Total Adults Arrested']) * 100
arrests_2019df['Percent of Drug Felony in Overall Adult Arrests'] = (arrests_2019df['Drug Felony'] / arrests_2019df['Total Adults Arrested']) * 100
arrests_2019df['Percent of Overall Drug Offenses in Overall Adult Arrests'] = (arrests_2019df['Total Drug Offenses'] / arrests_2019df['Total Adults Arrested']) * 100

arrests_2023df = pd.read_csv("/content/2023nonnycdrugarrests.csv")
arrests_2023df = arrests_2023df[['County', 'Year', 'Total', 'Felony Total', 'Drug Felony', 'Drug Misdemeanor']]
arrests_2023df['Total Drug Offenses'] = arrests_2023df['Drug Felony'] + arrests_2023df['Drug Misdemeanor']
arrests_2023df = arrests_2023df.rename(columns={"Total": "Total Adults Arrested"})
arrests_2023df['Total Drug Offenses'] = arrests_2023df['Drug Felony'] + arrests_2023df['Drug Misdemeanor']
arrests_2023df['Percent of Drug Felony in Overall Felony'] = (arrests_2023df['Drug Felony'] / arrests_2023df['Felony Total']) * 100
arrests_2023df['Percent of Drug Misdemeanor in Overall Adult Arrests'] = (arrests_2023df['Drug Misdemeanor'] / arrests_2023df['Total Adults Arrested']) * 100
arrests_2023df['Percent of Drug Felony in Overall Adult Arrests'] = (arrests_2023df['Drug Felony'] / arrests_2023df['Total Adults Arrested']) * 100
arrests_2023df['Percent of Overall Drug Offenses in Overall Adult Arrests'] = (arrests_2023df['Total Drug Offenses'] / arrests_2023df['Total Adults Arrested']) * 100


# 1. Merge the DataFrames on 'County' for comparison:
merged_df = pd.merge(arrests_2019df, arrests_2023df, on='County', suffixes=('_2019', '_2023'))

# 2. Calculate percentage differences for each category:
categories = ['Percent of Drug Felony in Overall Felony',
              'Percent of Drug Misdemeanor in Overall Adult Arrests',
              'Percent of Drug Felony in Overall Adult Arrests',
              'Percent of Overall Drug Offenses in Overall Adult Arrests']

for category in categories:
    # Access columns with the correct suffixes using f-strings for clarity.
    merged_df[f'{category}_Diff'] = ((merged_df[f'{category}_2023'] - merged_df[f'{category}_2019']) / merged_df[f'{category}_2019']) * 100

# 3. View the results:
pprint(merged_df[['County'] + [f'{category}_Diff' for category in categories]])

          County  Percent of Drug Felony in Overall Felony_Diff  \
0         Albany                                     -42.819137   
1       Allegany                                     -41.294867   
2         Broome                                     -35.007062   
3    Cattaraugus                                     -36.675562   
4         Cayuga                                     -45.786096   
5     Chautauqua                                      -4.210244   
6        Chemung                                     -43.838436   
7       Chenango                                     -13.503879   
8        Clinton                                     -32.849516   
9       Columbia                                     -43.606557   
10      Cortland                                     -10.707071   
11      Delaware                                     -52.277132   
12      Dutchess                                     -26.942711   
13          Erie                                     -36.50471

# Processing SEE Data (Dani)
It's not the most readable in the dataframe, so let's separate it and work some magic.

# 2023 SEE Data
I tried doing 2019 SEE data, was confused why I was getting zeros, then realized no one had SEE classifications at that time because the SEE classification went into place in 2021.

In [ ]:
# Now let's gather info from the SEE categories.
# All_data_2023['SEE Category'].unique()

In [ ]:
# All of these classifications are list-like, so use string (separate after comma) and explode function to turn them into their own rows.
exploded_categories = all_data_2023['SEE Category'].str.split(', ').explode()
unique_categories = exploded_categories.unique()
print(unique_categories)

[nan 'Minority-Owned Business' 'Women-Owned Business' 'Extra Priority'
 'Individuals from a community disproportionately impacted by the enforcement of cannabis prohibition'
 'Service-Disabled Veteran-Owned Business' 'Distressed Farmer']


In [ ]:
categories = {
    "Women-Owned Business": "Women-Owned Business",
    "Extra Priority": "Extra Priority",
    "Individuals from a community disproportionately impacted by the enforcement of cannabis prohibition": "Individuals from a community disproportionately impacted by the enforcement of cannabis prohibition",  # Original name
    "Service-Disabled Veteran-Owned Business": "Service-Disabled Veteran-Owned Business",
    "Distressed Farmer": "Distressed Farmer",
}

results = []

for category_name, category_column in categories.items():
    # The category_column is now the category_name itself, representing the column in all_data_2023.
    # Change: This line is modified to correctly filter using the category_column.
    filtered_businesses = all_data_2023[all_data_2023['SEE Category'].str.contains(category_column, na=False)]

    num_businesses = len(filtered_businesses)

    results.append([category_name, num_businesses])

SEE2023total_df = pd.DataFrame(results, columns=['Category', 'Number of Businesses'])
SEE2023total_df

,Category,Number of Businesses
0,Women-Owned Business,137
1,Extra Priority,12
2,Individuals from a community disproportionatel...,24
3,Service-Disabled Veteran-Owned Business,17
4,Distressed Farmer,4


In [ ]:
# This encodes the data from the SEE List, using lambda to define a function. Lambda x represents the individual value in the SEE categories list (which I already know exists). Category in x uses lambda x as a framework to check if the category is inside the list, returning 'true' if present and 'false' is absent. The last part, 'if isinstance(x, list) else false' ensures x is actually a list and if not assigns 'false' to a new column.
for category in unique_categories:
  all_data_2023[category] = all_data_2023['SEE Category'].apply(lambda x: category in x if isinstance(x, list) else False)

In [ ]:
all_data_2023

,License Number,License Type,License Type Code,License Status,License Status Code,Issued Date,Effective Date,Expiration Date,SEE Category,Entity Name,...,Percent of Drug Misdemeanor in Overall Adult Arrests,Percent of Drug Felony in Overall Adult Arrests,Percent of Overall Drug Offenses in Overall Adult Arrests,NaN,Minority-Owned Business,Women-Owned Business,Extra Priority,Individuals from a community disproportionately impacted by the enforcement of cannabis prohibition,Service-Disabled Veteran-Owned Business,Distressed Farmer
0,OCM-RETL-24-000123,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/10/2024 12:00:00 AM,07/10/2024 12:00:00 AM,07/10/2026 12:00:00 AM,NaN,10415 Jamaica LLC,...,7.223337,3.920358,11.143695,False,False,False,False,False,False,False
1,OCM-RETL-24-000138,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,07/16/2024 12:00:00 AM,07/16/2024 12:00:00 AM,07/16/2026 12:00:00 AM,NaN,1650 CRANE STREET DISPENSARY LLC,...,2.488038,1.913876,4.401914,False,False,False,False,False,False,False
2,OCM-RETL-24-000081,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,NaN,1WILDE1 LLC,...,4.196328,4.046459,8.242788,False,False,False,False,False,False,False
3,OCM-RETL-24-000267,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,12/19/2024 12:00:00 AM,12/19/2024 12:00:00 AM,12/19/2026 12:00:00 AM,NaN,296 Retail Venture LLC,...,4.230394,1.854865,6.085259,False,False,False,False,False,False,False
4,OCM-RETL-24-000082,Adult-Use Retail Dispensary License,OCMRETL,Active,LICACT,05/13/2024 12:00:00 AM,05/13/2024 12:00:00 AM,05/13/2026 12:00:00 AM,"Minority-Owned Business, Women-Owned Business",3807 harlem Cannabis LLC,...,6.815955,4.835549,11.651505,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,OCM-CAURD-24-000138,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,05/03/2024 12:00:00 AM,05/03/2024 12:00:00 AM,05/03/2026 12:00:00 AM,NaN,Valley Greens Ltd.,...,5.582950,2.799833,8.382783,False,False,False,False,False,False,False
436,OCM-CAURD-24-000198,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,08/23/2024 12:00:00 AM,08/23/2024 12:00:00 AM,08/23/2026 12:00:00 AM,NaN,Vedaleaf LLC,...,5.239521,2.929855,8.169376,False,False,False,False,False,False,False
437,OCM-CAURD-24-000124,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,04/12/2024 12:00:00 AM,04/12/2024 12:00:00 AM,04/12/2026 12:00:00 AM,NaN,Westchester Harvesting Company LLC,...,5.582950,2.799833,8.382783,False,False,False,False,False,False,False
438,OCM-CAURD-23-000006,Adult-Use Conditional Retail Dispensary License,OCMCAURD22,Active,LICACT,03/10/2023 12:00:00 AM,03/10/2023 12:00:00 AM,03/10/2025 12:00:00 AM,NaN,WilliamJane Corporation,...,6.332454,2.550572,8.883026,False,False,False,False,False,False,False


In [ ]:
import pandas as pd

categories = {
    "Women-Owned Business": "Women-Owned Business",
    "Extra Priority": "Extra Priority",
    "Individuals from a community disproportionately impacted by the enforcement of cannabis prohibition": "Individuals from a community disproportionately impacted by the enforcement of cannabis prohibition",  # Original name
    "Service-Disabled Veteran-Owned Business": "Service-Disabled Veteran-Owned Business",
    "Distressed Farmer": "Distressed Farmer",
}

all_counties = all_data_2023['County'].unique()  # Get unique counties

results = []

for county in all_counties:  # Loop through each county
    for category_name, category_column in categories.items():  # Loop through categories
        filtered_businesses = all_data_2023[
            (all_data_2023['County'] == county) &
            (all_data_2023['SEE Category'].str.contains(category_column, na=False))
        ]
        num_businesses = len(filtered_businesses)
        results.append([county, category_name, num_businesses])

SEE2023bycounty_df = pd.DataFrame(results, columns=['County', 'Category', 'Number of Businesses'])
SEE2023bycounty_df

,County,Category,Number of Businesses
0,Orange,Women-Owned Business,7
1,Orange,Extra Priority,0
2,Orange,Individuals from a community disproportionatel...,2
3,Orange,Service-Disabled Veteran-Owned Business,0
4,Orange,Distressed Farmer,1
...,...,...,...
245,Yates,Women-Owned Business,0
246,Yates,Extra Priority,0
247,Yates,Individuals from a community disproportionatel...,0
248,Yates,Service-Disabled Veteran-Owned Business,0
